In [ ]:
#%matplotlib inline
import os 
import argparse
import random

random.seed()
os.environ["THEANO_FLAGS"]  = "mode=FAST_RUN,device=gpu0,floatX=float32,lib.cnmem=1"
parser = argparse.ArgumentParser(description='Train model')                                                 
parser.add_argument('--backend', type=str, default="theano", help="theano or tensorflow")                      
parser.add_argument('--generator', type=str, default="upsampling", help="upsampling or deconv or subpixel")    
parser.add_argument('--discriminator', type=str, default="discriminator", help="discriminator discriminator_resnet")
parser.add_argument('--dset', type=str, default="mnist", help="mnistM or washington_vandal50k or washington_vandal12classes")             
parser.add_argument('--img_dim', default=64, type=int, help="Image width == height")                           
parser.add_argument('--nb_epoch', default=10000, type=int, help="Number of epochs")                              
parser.add_argument('--batch_size', default=32, type=int, help='Batch size for training purpose; for testing is hardcoded at 32')                                   
parser.add_argument('--n_batch_per_epoch', default=100, type=int, help="Number of batch per epochs")           
parser.add_argument('--bn_mode', default=2, type=int, help="Batch norm mode")                                  
parser.add_argument('--noise_dim', default=100, type=int, help="noise sampler dimension")                      
parser.add_argument('--noise_scale', default=0.5, type=float, help="noise sampler variance")                   
parser.add_argument('--disc_iterations', default=5, type=int, help="Number of discriminator iterations")
parser.add_argument('--clamp_lower', default=-100, type=float, help="Clamp weights below this value")         
parser.add_argument('--clamp_upper', default=100, type=float, help="Clamp weights above this value")          
parser.add_argument('--opt_D', type=str, default="Adam", help="Optimizer for the discriminator")            
parser.add_argument('--opt_G', type=str, default="Adam", help="Optimizer for the generator")                
parser.add_argument('--lr_D', type=float, default=1E-4, help="learning rate for the discriminator")            
parser.add_argument('--lr_G', type=float, default=1-4, help="learning rate for the generator")                
parser.add_argument('--use_mbd', action="store_true", help="use mini batch disc")
parser.add_argument('--deterministic', action="store_true", help="remove the noise input")
parser.add_argument('--inject_noise', action="store_true", help="inject noise into each layer")
parser.add_argument('--model', type=str, default="lsgan", help="wgan or lsgan") 
parser.add_argument('--no_supertrain', action="store_true", help="no discriminator supertraining") 
parser.add_argument('--monsterClass', action="store_true", help="Discriminator with 2x classes")
parser.add_argument('--resume', action="store_true", help="load GAN weights from previous train")#lsgan_VandWash1 is the name for the run Vand Wash no deterministic, no monsterclass
parser.add_argument('--name', type=str, default="VandToVand_N_class_deterministic_genLabelFixed", help="name of the run, used for saving and loading weights")
parser.add_argument('--wd', type=float, default=0.0, help="weight decay for generator/discriminator")

###simulating the main.py calling with corresponding parameters:
args = parser.parse_args('--resume --dset Vand_Vand_12class_LMDB --deterministic --monsterClass --disc_iterations 5 --no_supertrain --wd 1E-5 --lr_D 1E-4 --lr_G 1E-4'.split())
######
print args
assert args.dset in ["mnistM","washington_vandal50k","washington_vandal12classes","washington_vandal12classesNoBackground","Wash_Vand_12class_LMDB","Vand_Vand_12class_LMDB"]                                                      
assert args.opt_G in ["RMSprop", "SGD", "Adam", "AdamWithWeightnorm"], "Unsupported optimizer"                 
assert args.opt_D in ["RMSprop", "SGD", "Adam", "AdamWithWeightnorm"], "Unsupported optimizer" 

# Set the backend by modifying the env variable                                                                
if args.backend == "theano":                                                                                   
    os.environ["KERAS_BACKEND"] = "theano"                                                                     
elif args.backend == "tensorflow":                                                                             
    os.environ["KERAS_BACKEND"] = "tensorflow"  
import keras.backend as K
# manually set dim ordering otherwise it is not changed                                                        
if args.backend == "theano":                                                                                   
    image_dim_ordering = "th"                                                                                  
    K.set_image_dim_ordering(image_dim_ordering)                                                               
elif args.backend == "tensorflow":                                                                             
    image_dim_ordering = "tf"                                                                                  
    K.set_image_dim_ordering(image_dim_ordering)
import train_WGAN
# Set default params
d_params = {"generator": args.generator,
            "discriminator": args.discriminator,
            "dset": args.dset,
            "img_dim": args.img_dim,
            "nb_epoch": args.nb_epoch,
            "batch_size": args.batch_size,
            "n_batch_per_epoch": args.n_batch_per_epoch,
            "bn_mode": args.bn_mode,
            "noise_dim": args.noise_dim,
            "noise_scale": args.noise_scale,
            "disc_iterations": args.disc_iterations,
            "clamp_lower": args.clamp_lower,
            "clamp_upper": args.clamp_upper,
            "lr_D": args.lr_D,
            "lr_G": args.lr_G,
            "opt_D": args.opt_D,
            "opt_G": args.opt_G,
            "use_mbd": args.use_mbd,
            "deterministic": args.deterministic,
            "image_dim_ordering": image_dim_ordering,
            "model": args.model,
            "no_supertrain": args.no_supertrain,
            "monsterClass": args.monsterClass,
            "resume": args.resume,
            "name": args.name,
            "inject_noise": args.inject_noise,
            "wd" : args.wd
            }

# Launch training
train_WGAN.train(**d_params)

Namespace(backend='theano', batch_size=32, bn_mode=2, clamp_lower=-100, clamp_upper=100, deterministic=True, disc_iterations=5, discriminator='discriminator', dset='Vand_Vand_12class_LMDB', generator='upsampling', img_dim=64, inject_noise=False, lr_D=0.0001, lr_G=0.0001, model='lsgan', monsterClass=True, n_batch_per_epoch=100, name='VandToVand_N_class_deterministic_genLabelFixed', nb_epoch=10000, no_supertrain=True, noise_dim=100, noise_scale=0.5, opt_D='Adam', opt_G='Adam', resume=True, use_mbd=False, wd=1e-05)


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)



Experiment parameters:
noise_dim 100
image_dim_ordering th
opt_D Adam
use_mbd False
discriminator discriminator
clamp_lower -100
generator upsampling
nb_epoch 10000
deterministic True
n_batch_per_epoch 100
disc_iterations 5
no_supertrain True
monsterClass True
clamp_upper 100
opt_G Adam
wd 1e-05
resume True
batch_size 32
dset Vand_Vand_12class_LMDB
name VandToVand_N_class_deterministic_genLabelFixed
noise_scale 0.5
bn_mode 2
lr_G 0.0001
img_dim 64
lr_D 0.0001
inject_noise False
model lsgan


